In [52]:
import pandas as pd
import numpy as np
import sqlite3

# 🔁 Upsert function
def upsert_to_sqlite(df, db_path="laptop_prices.db", table_name="laptops"):
    df = df.copy()
    for col in df.select_dtypes(include=['datetime64[ns]']).columns:
        df[col] = df[col].dt.strftime('%Y-%m-%d')

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    columns = df.columns.tolist()
    column_defs = ", ".join([
        f'"{col}" TEXT' if df[col].dtype == 'object' else
        f'"{col}" REAL' if np.issubdtype(df[col].dtype, np.number) else
        f'"{col}" TEXT'
        for col in columns
    ])
    cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            {column_defs},
            UNIQUE(asin, scrape_date)
        );
    """)

    placeholders = ", ".join(["?"] * len(columns))
    columns_joined = ", ".join([f'"{col}"' for col in columns])
    update_clause = ", ".join([f'"{col}"=excluded."{col}"' for col in columns if col not in ['asin', 'scrape_date']])

    insert_query = f"""
        INSERT INTO {table_name} ({columns_joined})
        VALUES ({placeholders})
        ON CONFLICT(asin, scrape_date) DO UPDATE SET
        {update_clause};
    """

    cursor.executemany(insert_query, df.values.tolist())
    conn.commit()
    conn.close()


def fetch_merged_data_from_sqlite(db_path="laptop_prices.db", table_name="laptops"):
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    conn.close()
    return df


def update_merged_data(new_data_path="cleaned_Data.csv", db_path="laptop_prices.db", table_name="laptops"):
    try:
        merged_df = fetch_merged_data_from_sqlite(db_path=db_path, table_name=table_name)
    except:
        merged_df = pd.DataFrame()

    new_df = pd.read_csv(new_data_path)

    if not merged_df.empty:
        merged_df['scrape_date'] = pd.to_datetime(merged_df['scrape_date'], errors='coerce')
    new_df['scrape_date'] = pd.to_datetime(new_df['scrape_date'], errors='coerce')

    combined_df = pd.concat([merged_df, new_df], ignore_index=True)
    combined_df = combined_df.drop_duplicates(subset=['asin', 'scrape_date'], keep='last')
    combined_df = combined_df.sort_values(by=['asin', 'scrape_date'])

    combined_df['extracted_price'] = pd.to_numeric(combined_df['extracted_price'], errors='coerce')
    combined_df['rating'] = pd.to_numeric(combined_df['rating'], errors='coerce')
    combined_df['previous_price'] = combined_df.groupby('asin')['extracted_price'].shift(1)
    combined_df['previous_scrape_date'] = combined_df.groupby('asin')['scrape_date'].shift(1)

    combined_df['price_difference'] = combined_df['extracted_price'] - combined_df['previous_price']
    combined_df['price_change_percent'] = (combined_df['price_difference'] / combined_df['previous_price']) * 100

    mask = combined_df['previous_price'].isna()
    combined_df.loc[mask, 'previous_price'] = combined_df.loc[mask, 'extracted_price']
    combined_df.loc[mask, 'price_difference'] = 0
    combined_df.loc[mask, 'price_change_percent'] = 0
    combined_df.loc[mask, 'previous_scrape_date'] = combined_df.loc[mask, 'scrape_date']

    combined_df['buy_now'] = np.where(
        (combined_df['price_change_percent'] < -5) & (combined_df['rating'] > 3.2),
        'Yes', 'No'
    )
    
    # 💡 Price stability calculation
    stability_df = combined_df.groupby('asin')['extracted_price'].std().reset_index()
    stability_df.rename(columns={'extracted_price': 'price_stability'}, inplace=True)

    # Debug print for stability_df
    print("🔍 Stability DF Preview:\n", stability_df.head())

       # Step 8: Merge price stability (left join with 'asin')
    combined_df = pd.merge(combined_df, stability_df, on='asin', how='left', suffixes=('', '_stability'))
    
    # Step 9: Handle missing price_stability
    if 'price_stability' not in combined_df.columns:
        print("⚠️ Warning: 'price_stability' column missing after merge. Filling with 0.")
        combined_df['price_stability'] = 0
    else:
        combined_df['price_stability'] = combined_df['price_stability'].fillna(0)
    
    # Step 10: Label stability
    combined_df['stability_label'] = np.where(
        combined_df['price_stability'] < 5, 'Stable', 'Unstable'
    )
    
    # Clean up: Drop any extra columns that were added accidentally (e.g., price_stability_stability)
    extra_cols = [col for col in combined_df.columns if col.endswith('_stability') and col != 'price_stability']
    combined_df.drop(columns=extra_cols, inplace=True)

    combined_df = combined_df.sort_values(by=['asin', 'scrape_date'])

    upsert_to_sqlite(combined_df, db_path=db_path, table_name=table_name)

    print("✅ Merged data updated and saved to database.")
update_merged_data()

🔍 Stability DF Preview:
          asin  price_stability
0  B06XK8JXVX              NaN
1  B0751N2Y78              NaN
2  B07B6KYTXF              NaN
3  B07CCM5X58              NaN
4  B07FN6B1SV              NaN
✅ Merged data updated and saved to database.


In [53]:
import sqlite3

# Connect to the database
conn = sqlite3.connect("laptop_prices.db")
cursor = conn.cursor()

# List all tables
print("📂 Tables in database:")
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

# Describe table structure
print("\n📐 Structure of 'laptops' table:")
cursor.execute("PRAGMA table_info(laptops);")
for row in cursor.fetchall():
    print(row)

# Preview first 5 rows
print("\n🔍 First 5 rows of 'laptops' table:")
cursor.execute("SELECT * FROM laptops LIMIT 5;")
for row in cursor.fetchall():
    print(row)

conn.close()


📂 Tables in database:
[('laptops',)]

📐 Structure of 'laptops' table:
(0, 'Unnamed: 0', 'INTEGER', 0, None, 0)
(1, 'rating', 'REAL', 0, None, 0)
(2, 'reviews', 'INTEGER', 0, None, 0)
(3, 'extracted_price', 'REAL', 0, None, 0)
(4, 'asin', 'TEXT', 0, None, 1)
(5, 'title', 'TEXT', 0, None, 0)
(6, 'link_clean', 'TEXT', 0, None, 0)
(7, 'thumbnail', 'TEXT', 0, None, 0)
(8, 'delivery', 'TEXT', 0, None, 0)
(9, 'scrape_date', 'TEXT', 0, None, 2)
(10, 'display_size', 'TEXT', 0, None, 0)
(11, 'ram', 'TEXT', 0, None, 0)
(12, 'disk_size', 'TEXT', 0, None, 0)
(13, 'operating_system', 'TEXT', 0, None, 0)
(14, 'previous_price', 'REAL', 0, None, 0)
(15, 'previous_scrape_date', 'TEXT', 0, None, 0)
(16, 'price_difference', 'REAL', 0, None, 0)
(17, 'price_change_percent', 'REAL', 0, None, 0)
(18, 'buy_now', 'TEXT', 0, None, 0)
(19, 'price_stability', 'REAL', 0, None, 0)
(20, 'stability_label', 'TEXT', 0, None, 0)

🔍 First 5 rows of 'laptops' table:
(217, 4.2, 518, 648.88, 'B06XK8JXVX', 'Dell 15 3530 Lapto

In [65]:
import sqlite3
conn = sqlite3.connect('laptop_prices.db')
df = pd.read_sql("SELECT * FROM laptops", conn)




,Unnamed: 0,rating,reviews,extracted_price,asin,title,link_clean,thumbnail,delivery,scrape_date,...,ram,disk_size,operating_system,previous_price,previous_scrape_date,price_difference,price_change_percent,buy_now,price_stability,stability_label
0,217,4.200000,518.000000,648.88,B06XK8JXVX,Dell 15 3530 Laptop - Business & Home Student ...,https://www.amazon.com/Dell-15-3530-Laptop-Blu...,https://m.media-amazon.com/images/I/810XxIFk2l...,FREE delivery,2025-07-27,...,4 GB,1 TB,Windows 11 Pro,648.88,2025-07-27,0.0,0.0,No,0.0,Stable
1,73,4.000000,2220.000000,166.49,B0751N2Y78,Apple Macbook Air 2017 with 1.8GHz Intel Core ...,https://www.amazon.com/Apple-Macbook-13-inch-S...,https://m.media-amazon.com/images/I/91wYB53Y4a...,FREE delivery,2025-07-27,...,8 GB,128 GB,Mac OS,166.49,2025-07-27,0.0,0.0,No,0.0,Stable
2,61,4.700000,17.000000,529.00,B07B6KYTXF,"Dell 15.6"" Laptop Computer for Home and Studen...",https://www.amazon.com/Dell-Computer-Processor...,https://m.media-amazon.com/images/I/7179QXhArp...,FREE delivery,2025-08-01,...,12 GB,1 TB,Windows 11 Pro,529.00,2025-08-01,0.0,0.0,No,0.0,Stable
3,212,4.000000,1040.000000,179.00,B07CCM5X58,Apple MacBook Air MJVM2LL/A 11.6-Inch Laptop (...,https://www.amazon.com/Apple-11-6-Inch-Integra...,https://m.media-amazon.com/images/I/71N+DK0pEa...,FREE delivery,2025-08-01,...,4 GB,128 GB,macos_10_10_yosemite,179.00,2025-08-01,0.0,0.0,No,0.0,Stable
4,266,4.400000,126.000000,193.90,B07FN6B1SV,Dell Latitude 5590 Business Laptop | 15.6in HD...,https://www.amazon.com/Dell-Latitude-Business-...,https://m.media-amazon.com/images/I/81iSol1oyC...,FREE delivery,2025-08-01,...,8 GB,256 GB,Windows 10 Pro,193.90,2025-08-01,0.0,0.0,No,0.0,Stable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,405,5.000000,5.000000,399.99,B0FJ1QCPBF,"ACEMAGIC AX17 Laptop,17.3"" Laptop with Quad Co...",https://www.amazon.com/dp/B0FJ1QCPBF/,https://m.media-amazon.com/images/I/710iQwaYHb...,FREE delivery,2025-08-01,...,Info not available,512 GB,Windows 11,399.99,2025-08-01,0.0,0.0,No,0.0,Stable
563,282,5.000000,4.000000,489.99,B0FJ879N4G,"2025 Laptop with AMD Ryzen7(Up to 4.3GHZ,8C/16...",https://www.amazon.com/dp/B0FJ879N4G/,https://m.media-amazon.com/images/I/71WGMosroO...,FREE delivery,2025-07-27,...,Info not available,512 GB,Windows 11,489.99,2025-07-27,0.0,0.0,No,0.0,Stable
564,379,4.274584,290.850356,489.99,B0FJ879N4G,"2025 Laptop with AMD Ryzen7(Up to 4.3GHZ,8C/16...",https://www.amazon.com/dp/B0FJ879N4G/,https://m.media-amazon.com/images/I/71WGMosroO...,FREE delivery,2025-08-01,...,Info not available,512 GB,Windows 11,489.99,2025-07-27,0.0,0.0,No,0.0,Stable
565,262,5.000000,11.000000,839.00,B0FJF88GFW,Lenovo Laptop Computer PC IdeaPad Slim 3 for S...,https://www.amazon.com/dp/B0FJF88GFW/,https://m.media-amazon.com/images/I/71x6eHTfln...,FREE delivery,2025-08-01,...,40 GB,1 TB,Windows 11 Pro,839.00,2025-08-01,0.0,0.0,No,0.0,Stable
